In [1]:
# I could not install Spark on local PyCharm application. 
# That's why I am using Colab as you already suggested in the lecture.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install specific Java and Spark for Python.
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 

In [3]:
# Check to see if I installed Spark correctly or not.
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [16]:
# Inspect earthquake dataset.
# xM, verilen magnitüd (MD, ML, Mw, Ms, Mb) değerleri içerisindeki en büyük değerdir.
eqRDD = sc.textFile("/content/drive/MyDrive/Colab Notebooks/BDP/Datasets-20210410/EarthquakeDataset.txt")
print(eqRDD.take(2))

eqRDD  = eqRDD.filter(lambda line: "Event ID" not in line) # Filter the header

def split_and_get_important_columns(line):
    info = line.split("\t")
    event_no = info[0]

    # put date into a good format for processing later
    date = info[2]
    data = date.split(".")
    year = int(data[0])
    month = int(data[1])
    day = int(data[2])

    # put date into a good format for processing later
    time = info[3]
    time_data = time.split(":")
    hours = int(time_data[0])
    mins = int(time_data[1])
    sec_data = time_data[2].split(".")
    secs = int(sec_data[0])
    msecs = int(sec_data[1])

    lat = float(info[4])
    longt = float(info[5])
    magn = float(info[7])
    loc = info[-1]
    # I keep event_no, and location for easily identifying the foreshocks and aftershocks
    return (event_no, year, month, day, hours, mins, secs, msecs, lat, longt, magn, loc)

# Since I did not get any error in above line, it means there is no empty line in the dataset. If it exists, I would apply preprocessing.
eqRDD = eqRDD.map(lambda x: split_and_get_important_columns(x))
# Filter out wrong values for latitude and longitude if they exit.
eqRDD = eqRDD.filter(lambda line: -90<=line[-4]<=90)
eqRDD = eqRDD.filter(lambda line: -180<=line[-3]<=180)
# Filter between 1990 and 2019
eqRDD = eqRDD.filter(lambda line: 1990<=line[1]<=2019) # required in the question
topRDD = eqRDD.sortBy(lambda x: x[-2], ascending=False) # sort by biggest magnitude value

# Extract top ten by filtering according to magnitude value. Lowest magnitude among top ten earthquakes is 6.2.
topRDD = topRDD.filter(lambda x: x[-2]>=6.2) # luckily we do not have more than one earthquake with 6.2 magnitude in the dataset
print("Top 10 earthquakes between 1990 and 2019:")
print(topRDD.collect())
# Take a cartesian
cartRDD = topRDD.cartesian(eqRDD)
print("-----------------------------------------")
print(cartRDD.take(4))

['No    \tEvent ID\tDate\tOrigin Time\tLatitude\tLongitude\tDepth(km)\txM\tMD\tML\tMw\tMs\tMb\tType\tLocation', '000001\t20201231005127\t2020.12.31\t00:51:27.94\t37.8435\t26.7775\t008.0\t3.8\t0.0\t3.8\t3.7\t0.0\t0.0\tKe\tEGE DENIZI']
Top 10 earthquakes between 1990 and 2019:
[('009260', 1999, 8, 17, 0, 1, 37, 60, 40.76, 29.97, 7.4, 'BASISKELE (KOCAELI) [North East  2.0 km]'), ('004249', 2011, 10, 23, 10, 41, 21, 1, 38.7212, 43.411, 7.2, 'YEMLICE- (VAN) [North West  1.5 km]'), ('009044', 1999, 11, 12, 16, 57, 20, 80, 40.74, 31.21, 7.2, 'UGUR- (DUZCE) [North East  0.3 km]'), ('011247', 1992, 3, 13, 17, 18, 39, 40, 39.72, 39.63, 6.8, 'GUNEBAKAN- (ERZINCAN) [South West  1.7 km]'), ('001728', 2017, 7, 20, 22, 31, 9, 66, 36.9693, 27.4057, 6.6, 'GOKOVA KORFEZI (AKDENIZ)'), ('005473', 2008, 7, 15, 3, 26, 33, 58, 35.7948, 27.8798, 6.4, 'AKDENIZ'), ('007882', 2003, 5, 1, 0, 27, 4, 40, 39.01, 40.46, 6.4, 'KURTULUS- (BINGOL) [South West  4.3 km]'), ('001815', 2017, 6, 12, 12, 28, 37, 53, 38.8468, 

In [17]:
from geopy import distance
from datetime import datetime

def calculate_dist_and_time_interval(sample):
  # Do not consider the same samples by checking unique event no in the dataset 
  loc1 = sample[0][8:10]
  loc2 = sample[1][8:10]  
  dist = distance.distance(loc1, loc2).km # in km
  dist = round(dist, 2) # round it to 2 decimal points 
  if dist <= 20.0:
    dt_object1 = datetime(sample[0][1], sample[0][2], sample[0][3], hour=sample[0][4], minute=sample[0][5], second=sample[0][6], microsecond=sample[0][7])
    dt_object2 = datetime(sample[1][1], sample[1][2], sample[1][3], hour=sample[1][4], minute=sample[1][5], second=sample[1][6], microsecond=sample[1][7])
    hour_diff = (dt_object2 - dt_object1).total_seconds() / 3600.0
    if abs(hour_diff) <= 24.0:
      # I already do not consider tuple with same cases. That's why I can incude 0 hour difference here. Or leave it without including.
      # Assuming that you want us the aftershock and foreshock list with specific top earthquake that causes. I will add event_no of resulting earthquake to the lists.
      if hour_diff > 0:
        # Aftershock case
        return (1, sample[1], ("Resulting earthquake no:", sample[0][0]))
      elif hour_diff < 0:
        # Foreshocks case
        return (-1, sample[1], ("Resulting earthquake no:", sample[0][0]))

  return (0, sample[0]) # unnessary part for the problem

cartRDD = cartRDD.filter(lambda x: x[0][0] != x[1][0]) # filter same equartquakes in the cartesian product
cartRDD = cartRDD.map(lambda x: calculate_dist_and_time_interval(x))
print(cartRDD.take(4))

[(0, ('009260', 1999, 8, 17, 0, 1, 37, 60, 40.76, 29.97, 7.4, 'BASISKELE (KOCAELI) [North East  2.0 km]')), (0, ('009260', 1999, 8, 17, 0, 1, 37, 60, 40.76, 29.97, 7.4, 'BASISKELE (KOCAELI) [North East  2.0 km]')), (0, ('009260', 1999, 8, 17, 0, 1, 37, 60, 40.76, 29.97, 7.4, 'BASISKELE (KOCAELI) [North East  2.0 km]')), (0, ('009260', 1999, 8, 17, 0, 1, 37, 60, 40.76, 29.97, 7.4, 'BASISKELE (KOCAELI) [North East  2.0 km]'))]


In [20]:
aftershockRDD = cartRDD.filter(lambda x: x[0]==1) # filter aftershocks
aftershockRDD = aftershockRDD.map(lambda x: x[1:3]) # extract necessary information
aftershockRDD = aftershockRDD.sortBy(lambda x: x[0][0]) # sort by shock eqarthquake no
foreshockRDD = cartRDD.filter(lambda x: x[0]==-1) # filter foreshocks
foreshockRDD = foreshockRDD.map(lambda x: x[1:3]) # extract necessary information
foreshockRDD = foreshockRDD.sortBy(lambda x: x[0][0]) # sort by shock eqarthquake no
print("List of foreshocks (within 20 km and in 24 hours) for top ten earthquakes between 1990 and 2019:")
print(foreshockRDD.collect())
print("List of aftershocks (within 20 km and in 24 hours)for top ten earthquakes between 1990 and 2019:")
print(aftershockRDD.collect())

List of foreshocks (within 20 km and in 24 hours) for top ten earthquakes between 1990 and 2019:
[]
List of aftershocks (within 20 km and in 24 hours)for top ten earthquakes between 1990 and 2019:
[(('001648', 2017, 7, 21, 21, 51, 1, 1, 36.9092, 27.6078, 3.9, 'GOKOVA KORFEZI (AKDENIZ)'), ('Resulting earthquake no:', '001728')), (('001650', 2017, 7, 21, 17, 9, 46, 28, 36.9465, 27.2537, 4.9, 'AKYARLAR-BODRUM (MUGLA) [South West  4.5 km]'), ('Resulting earthquake no:', '001728')), (('001651', 2017, 7, 21, 16, 55, 24, 70, 36.9867, 27.334, 3.8, 'AKYARLAR-BODRUM (MUGLA) [North East  4.0 km]'), ('Resulting earthquake no:', '001728')), (('001652', 2017, 7, 21, 14, 21, 55, 45, 36.9397, 27.4797, 3.9, 'GOKOVA KORFEZI (AKDENIZ)'), ('Resulting earthquake no:', '001728')), (('001653', 2017, 7, 21, 12, 5, 24, 49, 36.9125, 27.531, 3.6, 'GOKOVA KORFEZI (AKDENIZ)'), ('Resulting earthquake no:', '001728')), (('001654', 2017, 7, 21, 11, 6, 38, 92, 36.931, 27.5042, 3.6, 'GOKOVA KORFEZI (AKDENIZ)'), ('Resul